# Display metrics related to interesting GitHub repo activity using Pyspark

## TODO
* Save and restore datasets to S3
* Other stuff

In [ ]:
import pandas as pd
import matplotlib.pyplot as plot

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

In [ ]:
conf = SparkConf().setAppName("github-stats")

sc = SparkContext(conf=conf)
sql = SQLContext(sc)

In [ ]:
# Original data using sqlite
# import sqlite3
# conn = sqlite3.connect('data/github.db')
# data = pd.read_sql_query("""
# SELECT strftime('%Y-%m-01', date) AS date, organization, repository,
# MAX(stargazers) AS stargazers, MAX(forks) AS forks
# FROM github_stats
# GROUP BY organization, repository, strftime('%Y-%m', date)
# """, conn)

In [ ]:
mystats = sql.read.csv('s3a://gregoryg/datasets/github_stats/github_stats.csv.gz', inferSchema=True, header=True)

In [ ]:
mystats.createTempView('github_stats')

In [ ]:
data = sql.sql("""
select
  date_format(`date`, "yyyy-MM") AS date, 
  organization, 
  repository, 
  MAX(stargazers) AS stargazers, 
  MAX(forks) AS forks 
  FROM github_stats 
  GROUP BY organization, repository, date_format(`date`, "yyyy-MM")
""").toPandas()

In [ ]:
# data

In [ ]:
def subset(org_and_repo):
    org, repo = org_and_repo.split('/')
    return data.loc[(data.organization == org) & (data.repository == repo)]

def merge_repos(repos, metric='stargazers'):
    data = pd.DataFrame(columns=['Date'])
    for repo in repos:
        repo_data = subset(repo)
        repo_data = repo_data[['date', metric]].rename(columns={metric: repo, 'date': 'Date'})
        data = data.merge(repo_data, on='Date', how='outer')
    return data.sort_values(by='Date')

def plot_repos(title, repos, metric='stargazers'):
    plot_data = merge_repos(repos, metric)
    plot_data.plot(x='Date', y=repos, title=title, figsize=(15,10))

def plot_all_repos(title, org, metric='stargazers'):
    repo_data = data.loc[data.organization == org]
    repos = [org + '/' + repo for repo in repo_data.repository.unique()]
    plot_repos(title, repos, metric)

In [ ]:
repos = [
    'tensorflow/tensorflow',
    'apache/incubator-mxnet',
    'Microsoft/CNTK',
    'BVLC/caffe',
    'keras-team/keras',
    'Theano/Theano',
    'amzn/amazon-dsstne',
    'pytorch/pytorch'
]
plot_repos('Machine Learning', repos)

In [ ]:
repos = [
    'kubeflow/kubeflow',
    'tensorflow/tfx'
]
plot_repos('Machine Learning Platforms', repos)

In [ ]:
repos = [
    'apache/spark',
    'apache/ignite',
    'apache/flink',
    'apache/beam',
    'apache/storm',
    'apache/samza'
]
plot_repos('Big Data', repos)

In [ ]:
repos = [
    'fnproject/fn',
    'openfaas/faas',
    'apache/incubator-openwhisk',
    'fission/fission',
    'apex/apex',
    'serverless/serverless',
    'kubeless/kubeless',
    'knative/serving'
]

plot_repos('Functions as a Service', repos)

In [ ]:
repos = [
    'mesosphere/marathon',
    'hashicorp/nomad',
    'kubernetes/kubernetes',
    'docker/swarm',
    'docker/swarmkit'
]

plot_repos('Container Orchestration', repos)

In [ ]:
repos = [
    'jupyter/notebook',
    'apache/zeppelin',
    'jupyterhub/jupyterhub'
]

plot_repos('Data Science Notebooks', repos)

In [ ]:
plot_all_repos('Kubernetes', 'kubernetes')

In [ ]:
plot_all_repos('Operator Framework', 'operator-framework')

In [ ]:
repos = [
    'operator-framework/operator-sdk',
    'kubernetes-sigs/kubebuilder',
    'kubernetes-sigs/controller-runtime',
    'kudobuilder/kudo',
    'bluek8s/kubedirector',
    'kubedb/operator',
    'rook/operator-kit',
    'GoogleCloudPlatform/metacontroller',
    'zalando-incubator/kopf'
]

plot_repos('Operator Tools', repos)

In [ ]:
# Get my awesome operators
!python3 awesome_operators.py > awesome_operators.txt

In [ ]:
awesome_repo_names = [l.rstrip('\n') for l in open('awesome_operators.txt').readlines()]

def name_and_stars(name):
    awesome_data = subset(name)
    max_stars = awesome_data['stargazers'].max()
    return (name, max_stars)

awesome_data = pd.DataFrame(map(name_and_stars, awesome_repo_names), columns=['repository', 'stargazers']).dropna()

n = 50
top_n_operators = awesome_data.sort_values(by='stargazers', ascending=False).head(n)
top_n_operators

In [ ]:
top_n_operators.plot(x='repository', y='stargazers', kind='bar', figsize=(20,10), title='Github stars for operators listed in awesome-operators')

In [ ]:
plot_all_repos('Kubernetes SIGs', 'kubernetes-sigs')

In [ ]:
most_stars = data.groupby(['organization', 'repository']).max().sort_values(['stargazers'])
most_stars

In [ ]:
plot_all_repos('Gardener', 'gardener')

In [ ]:
repos = [
    'istio/istio',
    'linkerd/linkerd2'
]
plot_repos('Service Mesh', repos)

In [ ]:
repos = [
    'operator-framework/operator-sdk',
    'kubernetes-sigs/kubebuilder',
    'kudobuilder/kudo',
    'bluek8s/kubedirector',
    'kubedb/operator',
    'rook/operator-kit',
    'GoogleCloudPlatform/metacontroller',
    'zalando-incubator/kopf'
]

plot_data = merge_repos(repos, 'forks')
plot_data

In [ ]:
# sc.stop()